In [ ]:
# 구글 드라이브에서 크롤링 한 파일 가져와서 라벨링 및 임베딩
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from sentence_transformers import SentenceTransformer
import chromadb
import openai
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

# Google Drive API 인증 설정
SERVICE_ACCOUNT_FILE = 'service_account.json'  # JSON 파일 경로
SCOPES = ['https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=credentials)

# 공유 드라이브의 폴더 ID 설정 (공유 드라이브 URL에서 추출)
shared_folder_id = "1GwbcEfpaCbcbkhBpXjR4GOO6UFVjceiz"  # 실제 공유 드라이브 ID로 대체

# SentenceTransformer 및 ChromaDB 초기화
model = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.PersistentClient()

# 기존 컬렉션 삭제 후 생성
if "crawled_data" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="crawled_data")
collection = client.create_collection(name="crawled_data")


# 라벨링 함수 수정
def label_document(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "당신은 문서를 분석하여 적절한 주제를 할당하는 어시스턴트입니다. 가능한 주제는 '주거', '취업', '문화활동', '금융'입니다. 각 문서를 분석하여 가장 관련이 높은 주제를 선택하세요."},
                {"role": "user", "content": f"다음 문서의 주제를 분석하고 적합한 주제를 단어로만 반환하세요:\n\n{content}"}
            ],
            model="gpt-4o",
        )
        label = response.choices[0].message.content
        print(response.choices[0].message.content)
        return label

    except Exception as e:
        print(f"라벨링 실패: {e}")
        return "기타"  # 실패 시 기본 라벨



# Google Drive에서 파일 목록 가져오기
def get_txt_files_from_drive(folder_id):
    query = f"'{folder_id}' in parents and mimeType='text/plain'"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    return results.get('files', [])

# Google Drive에서 파일 내용 가져오기
def download_file(file_id):
    request = drive_service.files().get_media(fileId=file_id)
    file_content = request.execute()
    return file_content.decode('utf-8')

# Google Drive의 텍스트 파일 처리
files = get_txt_files_from_drive(shared_folder_id)
for idx, file in enumerate(files):
    file_content = download_file(file['id']).strip()
    if file_content:  # 내용이 있는 경우만 처리
        # 라벨링
        label = label_document(file_content)
        if not label:  # 라벨이 None인 경우 처리
            print(f"파일 '{file['name']}'의 라벨 생성 실패로 저장되지 않았습니다.")
            continue  # 저장 작업 생략

        print(f"파일 '{file['name']}'에 대한 라벨: {label}")
        
        # 임베딩 생성
        try:
            embedding = model.encode([file_content])[0]
        except Exception as e:
            print(f"임베딩 생성 실패: {e}")
            continue  # 저장 작업 생략

        # 데이터 저장
        try:
            collection.add(
                documents=[file_content],
                embeddings=[embedding],
                metadatas=[{"label": label}],
                ids=[f"doc_{idx+1}"]
            )
            print(f"파일 '{file['name']}'의 내용을 저장했습니다.")
        except ValueError as e:
            print(f"데이터 저장 실패: {e}")

### 성공했을시 출력 결과입니다

취업
파일 'content_10.txt'에 대한 라벨: 취업
파일 'content_10.txt'의 내용을 저장했습니다.
주거
파일 'content_9.txt'에 대한 라벨: 주거
파일 'content_9.txt'의 내용을 저장했습니다.

원래는 문장으로 왜 이 라벨로 구분했는지 말했는데 라벨링도 같이 임베딩 해야하다보니 단어로만 반환하게 만들었습니다. 나중에 필요하다면 단어로도 반환하고 이유도 말해달라고 해서 따로 저장하면 될 듯하네요

그리고 중복 허용도 해야하고... 아직 갈길이 머네요...

In [ ]:
# chromaDB csv파일로 내보내는 코드
import csv

def export_chromadb_to_csv(collection, file_name="chromadb_data.csv"):
    try:
        # ChromaDB 컬렉션에서 모든 데이터 가져오기
        results = collection.get(include=["documents", "metadatas", "embeddings"])
        
        # 데이터의 ID는 별도로 관리되므로 수동으로 추가
        ids = collection.get()["ids"]
        
        with open(file_name, mode="w", encoding="utf-8", newline="") as csvfile:
            writer = csv.writer(csvfile)
            # CSV 헤더 작성
            writer.writerow(["ID", "Document", "Metadata", "Embedding"])
            
            # 데이터 작성
            for doc_id, document, metadata, embedding in zip(
                ids, results["documents"], results["metadatas"], results["embeddings"]
            ):
                writer.writerow([doc_id, document, metadata, embedding])
        
        print(f"ChromaDB 데이터가 '{file_name}'로 저장되었습니다.")
    except Exception as e:
        print(f"CSV로 내보내는 데 실패했습니다: {e}")

# CSV로 내보내기 실행
export_chromadb_to_csv(collection)
